In [3]:
!pip install -U accelerate datasets

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached xxhash-3.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (31 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metad

In [1]:
!pip install -q chromadb

In [12]:
import torch.nn as nn
import numpy as np
# import Datasets
import chromadb
from sentence_transformers import (
    SentenceTransformer, models, losses, util, InputExample, evaluation, SentenceTransformerTrainingArguments, SentenceTransformerTrainer
)
from accelerate import Accelerator
import glob
import os


# Basic functions the model can use

In [4]:
def getEmbeddingList(model, sentences):
  """ This function returns the sentence embeddings for a given document using the SentenceTransformer model and encapsulates them inside a list.

  @param model: SentenceTransformer: The model to be used for getting the embeddings.
  @param sentences: list: The list of sentences for which embeddings are to be calculated. """

  embeddings = model.encode(sentences)
  return embeddings.tolist()

In [5]:
def getModel() -> SentenceTransformer:
  """ This function creates a SentenceTransformer model using the 'sentence-transformers/all-MiniLM-L6-v2' base model. It utilizes accelerator to make use of multiple GPUs
  and adds a layer to get the sentence embeddings via mean pooling. This model will be used for training sbert's sentence embeddings. """

  accelerator = Accelerator()
  print(f"Using GPUs: {accelerator.num_processes}")

  # Get the base model to train
  word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')

  # Add layer to get "sentence embedding" (using mean pooling)
  pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
  model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
  return model

In [ ]:
# An initial list of actions that the AI can choose between
SET_OF_ACTIONS = ['new file', 'search web', 'search files', 'resize window', 'choose option', 'open file', 'close file', 'minimize window', 'maximize window', 'scroll up', 'scroll down', 'scroll left', 'scroll right', 'open', 'close', 'upload']
# SET_OF_ACTIONS = ['new file', 'search', 'resize window', 'choose option', 'scroll', 'open file', 'close file', 'minimize window', 'maximize window', 'scroll up', 'scroll down', 'scroll left', 'scroll right', 'copy', 'paste', 'cut', 'undo', 'redo', 'drag and drop', 'select', 'deselect', 'save', 'save as', 'open', 'close', 'upload']

In [16]:
client = chromadb.Client()

doc_collection = client.get_or_create_collection("docs")

In [8]:
model = getModel()

Using GPUs: 1


In [23]:
dir_path = "test/"  # Replace with the actual directory path

file_list = []
for file_name in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file_name)
    with open(file_path, 'r') as file:
        data = file.read()
        file_list.append(data)
        
print(file_list)

file_names = [file_name for file_name in os.listdir(dir_path)]
print(file_names)

['1. Exercise regularly and stay fit.\n2. Learn a new programming language.\n3. Read at least one book every month.\n4. Spend more time with family and friends.\n5. Travel to at least two new places.\n6. Save money and stick to a budget.\n7. Volunteer for a good cause.\n8. Improve my communication skills.\n9. Learn a musical instrument.\n10. Practice mindfulness and reduce stress.', 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n\ndef selection_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        min_idx = i\n        for j in range(i+1, n):\n            if arr[j] < arr[min_idx]:\n                min_idx = j\n        arr[i], arr[min_idx] = arr[min_idx], arr[i]\n\ndef insertion_sort(arr):\n    n = len(arr)\n    for i in range(1, n):\n        key = arr[i]\n        j = i-1\n        while j >= 0 and arr[j] > key:\n            arr[j+1] = arr[j

In [24]:
doc_collection.add(
    embeddings=
        getEmbeddingList(model, file_list)
    ,
    documents=file_names,
    ids=[f'id{i}' for i in range(len(file_list))],
)

Add of existing embedding ID: id0
Add of existing embedding ID: id1
Add of existing embedding ID: id2
Insert of existing embedding ID: id0
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2


### Fine tune this or look at online examples because current outputs are bad

In [22]:
# bad
input = "Python codes for web scraping"

query_result = doc_collection.query(
            query_embeddings=[getEmbeddingList(model, input)],
            n_results=1,
        )

print(query_result)

{'ids': [['id0']], 'distances': [[59.03951644897461]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['sorting.py']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [25]:
# bad
input = "new years resolutions"

query_result = doc_collection.query(
            query_embeddings=[getEmbeddingList(model, input)],
            n_results=1,
        )

print(query_result)

{'ids': [['id2']], 'distances': [[46.51013946533203]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['example.txt']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [ ]:
# Now we need to get a vocal dataset from hf
